In [83]:
from collections import defaultdict
import json
cs=set()
cin=defaultdict(str)
cni=defaultdict(str)
with open('DataCrawling/championInfo.json', 'r') as f:
    d = json.load(f)
    for champ, champ_dict in d['data'].items():
        cs.add(champ_dict['key'])
        cin[champ_dict['key']]=champ_dict['id']
        cni[champ_dict['id']]=champ_dict['key']
        
print(cin)

defaultdict(<class 'str'>, {'266': 'Aatrox', '103': 'Ahri', '84': 'Akali', '12': 'Alistar', '32': 'Amumu', '34': 'Anivia', '1': 'Annie', '22': 'Ashe', '136': 'AurelionSol', '268': 'Azir', '432': 'Bard', '53': 'Blitzcrank', '63': 'Brand', '201': 'Braum', '51': 'Caitlyn', '164': 'Camille', '69': 'Cassiopeia', '31': 'Chogath', '42': 'Corki', '122': 'Darius', '131': 'Diana', '119': 'Draven', '36': 'DrMundo', '245': 'Ekko', '60': 'Elise', '28': 'Evelynn', '81': 'Ezreal', '9': 'Fiddlesticks', '114': 'Fiora', '105': 'Fizz', '3': 'Galio', '41': 'Gangplank', '86': 'Garen', '150': 'Gnar', '79': 'Gragas', '104': 'Graves', '120': 'Hecarim', '74': 'Heimerdinger', '420': 'Illaoi', '39': 'Irelia', '427': 'Ivern', '40': 'Janna', '59': 'JarvanIV', '24': 'Jax', '126': 'Jayce', '202': 'Jhin', '222': 'Jinx', '145': 'Kaisa', '429': 'Kalista', '43': 'Karma', '30': 'Karthus', '38': 'Kassadin', '55': 'Katarina', '10': 'Kayle', '141': 'Kayn', '85': 'Kennen', '121': 'Khazix', '203': 'Kindred', '240': 'Kled', '9

In [84]:
import surprise as sp
import pandas as pd
import re

ul=[]
cl=[]
ml=[]
ucp=defaultdict(int)
for line in open("DataCrawling/playersInfo.txt"):
    data = line.strip().split('-')
    if len(data) != 3:
        summoner = data[-3].split(',')[-1]
        champ_list = data[-2]
        rank = data[-1]
    else:
        summoner = data[0]
        champ_list = data[1]
        rank = data[2]
    champ_list=champ_list.split(')')
    champ_list=[i.replace('(','').lstrip(',') for i in champ_list if i!='']
    if summoner=="81209212": print(champ_list)
    for i in champ_list:
        if i=='': continue
        p=i.split(',')
        cs.add(p[0])
        if int(p[1])>=7:
            ul.append(summoner)
            cl.append(p[0])
            ml.append(p[1])
            ucp[str(summoner)+'-'+str(p[0])]=2
        elif int(p[1])>=4:
            ul.append(summoner)
            cl.append(p[0])
            ml.append(p[1])
            ucp[str(summoner)+'-'+str(p[0])]=1
    for i in cs:
        if ucp[str(summoner)+'-'+str(i)]==0:
            ul.append(summoner)
            cl.append(i)
            ml.append(0)
        
            

['236,7,98025', '7,7,94126', '238,7,85924', '157,7,76986', '67,5,75892', '266,5,73301', '134,7,67321', '112,7,55761', '64,5,54737', '105,6,51842', '60,5,44983', '81,5,44104', '107,7,43529', '92,6,43301', '59,5,42943', '121,6,42680', '58,6,42519', '51,6,42363', '61,6,38955', '38,6,33190', '245,5,32518', '24,5,31683', '126,5,31359', '11,5,30832', '68,5,30464', '202,5,30285', '122,5,29843', '201,5,29786', '18,5,29714', '222,5,28063', '268,5,26894', '4,5,26158', '39,5,26019', '69,5,25678', '57,5,23246', '145,5,23119', '103,5,22255', '106,5,22235', '29,5,21726', '120,4,20544', '498,4,18079', '56,4,17415', '96,4,17242', '14,4,17159', '89,4,16145', '2,4,16034', '412,4,15297', '21,4,15277', '154,4,14809', '15,4,14174', '91,4,14053', '79,4,13883', '62,4,13103', '1,4,12960', '102,4,12833', '40,4,12604', '114,3,12176', '54,3,11845', '31,3,11824', '32,3,11054', '85,3,10935', '131,3,9768', '75,3,9696', '5,3,9669', '45,3,9570', '84,3,9546', '110,3,9127', '12,3,8797', '22,3,8436', '99,3,7885', '25,3,

In [87]:
import random
import numpy as np
#t=random.sample(range(0,1000000),200000)
#v=random.sample(range(0,1000000),200000)
testu=ul[:1000000]
testc=cl[:1000000]
testm=ml[:1000000]
pid="1"
pdi={"Xayah":"7","Kaisa":"7","Ezreal":"7","Braum":"5","Jhin":"5","Twitch":"5"}
for i in pdi:
    testu.append(pid)
    testc.append(cni[i])
    testm.append(pdi[i])
    ucp[pid+"-"+cni[i]]=(2 if int(pdi[i])>=7 else 1)
for i in cs:
    if ucp[pid+'-'+str(i)]==0:
        testu.append(pid)
        testc.append(i)
        testm.append(0)
mainDict={'ul':testu,'cl':testc,'ml':testm}
pm=pd.DataFrame(mainDict)
reader=sp.Reader(rating_scale=(0,7))
data=sp.Dataset.load_from_df(pm[['ul', 'cl', 'ml']],reader)
sim_options = {'name': 'pearson_baseline',
                'user_based': False 
                }
alg = sp.KNNBasic(sim_options=sim_options)
output=alg.fit(data.build_full_trainset())

st=0
t=0
f=0
sf=0
rec=[]
top=[(0,-1),(0,-1),(0,-1),(0,-1),(0,-1)]
pl=[]
select=1000005
for i in cs:
    p=alg.predict(testu[select],i).est
    pl.append(p)
for i in cs:
    p=alg.predict(testu[select],i)
    r=ucp[str(testu[select])+'-'+str(i)]
    th=max(pl)/2
    print(p)
    print(r)
    if p.est>th and r>=2:
        st+=1
    elif r>=2:
        sf+=1
    elif p.est>th and r>=1:
        t+=1
    elif r>=1:
        f+=1
    elif p.est>th:
        rec.append(cin[i])
    if p.est>top[0][0]:
        if r<1:
            top[0]=(p.est,i)
            top=sorted(top)
if len(rec)<5:
    rec=[cin[i[1]] for i in top]
print(rec)
print("Caught(Significant):"+str(st))
print("Caught:"+str(t))
print("Missed:"+str(f))
print("Missed:(Significant):"+str(sf))
print("Recall:"+str((st+t)/(st+t+f+sf)))

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
user: 1          item: 25         r_ui = None   est = 0.11   {'actual_k': 40, 'was_impossible': False}
0
user: 1          item: 107        r_ui = None   est = 0.17   {'actual_k': 40, 'was_impossible': False}
0
user: 1          item: 121        r_ui = None   est = 0.29   {'actual_k': 40, 'was_impossible': False}
0
user: 1          item: 16         r_ui = None   est = 0.12   {'actual_k': 40, 'was_impossible': False}
0
user: 1          item: 44         r_ui = None   est = 0.19   {'actual_k': 40, 'was_impossible': False}
0
user: 1          item: 516        r_ui = None   est = 0.00   {'actual_k': 40, 'was_impossible': False}
0
user: 1          item: 51         r_ui = None   est = 1.02   {'actual_k': 40, 'was_impossible': False}
0
user: 1          item: 7          r_ui = None   est = 0.25   {'actual_k': 40, 'was_impossible': False}
0
user: 1          item: 81         r_ui = No

In [50]:
example="(115,5,36488),(412,4,16670),(111,3,12345)"
example=example.split(')')
example=[i.replace('(','').lstrip(',') for i in example if i!='']
print(example)

['115,5,36488', '412,4,16670', '111,3,12345']


In [75]:
print(cin["498"])

Xayah
